<h1><center>Align Dragonfly Data</center></h1>

In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from tkinter import Tk, filedialog

import os
import h5py
import pickle
import numpy as np
import napari
from scipy import ndimage as ndi
from datetime import datetime
from PyImarisWriter import PyImarisWriter as PW
from imaris_ims_file_reader.ims import ims

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
#global variables
im_path = ' '
im_path_list = []
align_path = ' '
im_shape = 0
im_resolution = 0
im_dtype = 0
channel_names_org = []
tmat_xyz_list = []
tmat_xy_list = []
z_step = 1
results = []

image_3D = True
transform_3D = True
all_time = True
transform_choice = '3D'

time_slider = widgets.IntRangeSlider()

# colors for visualisation
vis_colors = ['magenta','red','green','blue']

In [4]:
def check_selected_files(im_path,align_path):
    
    global show_options_b
    
    im_test = 'ims' in im_path
    align_test = 'pkl' in align_path
    
    if (im_test and align_test):
        
        show_options_b.layout.visibility = 'visible'
        
    else: 
        
        show_options_b.layout.visibility = 'hidden'
        
#######################################################################################
#######################################################################################  
def get_metadata(im_path):
    
    im = ims(im_path)
    
    shape = im.shape
    resolution = im.resolution
    dtype = im.dtype
    
    return shape,resolution,dtype

#######################################################################################
#######################################################################################  

# function to center display
def centered(content):
    display(HTML(f"<div style='text-align:center'>{content}</div>"))

#######################################################################################
#######################################################################################    
    
# function to choose a file
def select_file(b):
        clear_output()                                        
        root = Tk()
        root.withdraw()                                        
        root.call('wm', 'attributes', '.', '-topmost', True)   
        b.files = filedialog.askopenfilename(multiple=True)    
        if len(b.files)>0:
            for f in b.files:
                centered(f) 
#######################################################################################
#######################################################################################
            
def select_file_movie(b):    
    
    global im_path
    global im_path_list
    
    with out_file:
        select_file(b)
        if len(b.files)>0:
            im_path = b.files[0]
            im_path_list = b.files
        else:
            im_path = ' '        
        check_selected_files(im_path,align_path)

#######################################################################################
#######################################################################################
        
def select_file_align(b):
    global align_path
    
    with out_align:
        select_file(b)
        if len(b.files)>0:
            align_path = b.files[0]
        else:
            align_path = ' '
        check_selected_files(im_path,align_path)

#######################################################################################
#######################################################################################
        
def show_options(b):    
    
    global im_shape
    global im_resolution
    global im_dtype
    global frame_num
    global channel_names_org
    global tmat_xyz_list
    global tmat_xy_list
    global z_step
    global image_3D 
    global transform_3D
    global time_slider
    
    #  get metadata of the data
    im_shape,im_resolution,im_dtype = get_metadata(im_path)
    
    # check if data are 2D or 3D
    image_3D = (im_shape[2] > 1)
    
    # get alignment details
    transform_3D = not('_2D_' in align_path)
    
    with open(align_path,'rb') as f:
        
        if transform_3D:
            
            channel_names_org,tmat_z_list,tmat_xy_list,tmat_xyz_list,z_step = pickle.load(f) 
            
        else: 
            channel_names_org,tmat_xy_list = pickle.load(f) 
        
    # build options menu
    create_options_menu()
    
    # create alignment button
    align_b = widgets.Button(description="Register channels")
    align_b.on_click(align_f_multi)
    align_box = widgets.HBox(children=[align_b],layout=box_layout_col)
    display(align_box)
    
#######################################################################################
#######################################################################################

def create_dropdown(channel_names_org,ch_number):

    dropdown = widgets.Dropdown(
        options=channel_names_org,
        value=channel_names_org[ch_number],
        layout = {'width':'148px'}
    )
    
    return dropdown

#######################################################################################
#######################################################################################

def create_options_menu():
    
    global time_range
    global transform_choice
    
    toggle_list=[]
    toggle_dict={}

    for ch_number in range(im_shape[1]):

        button_channel = widgets.ToggleButton(description=f'Channel {ch_number +1}', value=True)

        toggle_list.append(button_channel)

        toggle_dict[f'button_ch_{ch_number}'] = button_channel
        
    #######################################################################################

    dropdown_list=[]
    dropdown_dict = {}
    for ch_number in range(im_shape[1]):

        dropdown = create_dropdown(channel_names_org,ch_number)

        dropdown_list.append(dropdown)
        dropdown_dict[f'drop_ch_{ch_number}'] = dropdown

    #######################################################################################

    reference_toggle = widgets.ToggleButtons(
        options=[f'Channel {x+1}' for x in range(im_shape[1])],
        disabled=False
    )
    
    #######################################################################################    
    # create labels
    lab_wid = widgets.VBox([widgets.Label(value="Channels to use:"),widgets.Label(value="Settings:"),
                 widgets.Label(value="Anchor:")],layout = {'width':'300px'})


    #######################################################################################    
    # put all the elements of the interface in a common dictionary 

    toggle_dict.update(dropdown_dict)    
    toggle_dict['ref_toggle'] = reference_toggle

    #######################################################################################

    box_4 = widgets.HBox(toggle_list)
    box_5 = widgets.HBox(dropdown_list,layout=box_4.layout)
    box_6 = widgets.HBox(children=[reference_toggle],layout=box_layout_col)
    
    box_all = widgets.VBox([box_4,box_5,box_6],layout=box_layout_col)
    
    box_explain = widgets.HBox([lab_wid,box_all])

    interact_out = widgets.interactive_output(on_change, toggle_dict)
    display(box_explain,interact_out) 
    
    #######################################################################################
    
    if (transform_3D and image_3D):
        
        transform_b = widgets.ToggleButton(description="Transform in 3D",value=True)
        transform_b.observe(choose_3D_f,'value')
        transform_box = widgets.HBox(children=[transform_b],layout=box_layout_col)
        display(transform_box)
        
    elif (transform_3D and not(image_3D)):
        
        print('Using xy from the 3D transform.')
        transform_choice = '2D'
        
    elif (not(transform_3D) and image_3D):
        
        print('You selected a 2D transform. Only xy transformation will be applied.')
        transform_choice = '2D'  
        
    else:
        
        transform_choice = '2D'     
        

    #######################################################################################
    frame_num = im_shape[0]
    if frame_num > 1:
        
        all_range_b = widgets.ToggleButton(description="All Time Points",value=True)
        all_range_b.observe(choose_all_range_f,'value')
        
        #set slider
        time_slider.value=[0, frame_num]
        time_slider.min = 1
        time_slider.max = frame_num
        time_slider.disabled=True
        
        box_7 = widgets.HBox(children=[all_range_b,time_slider],layout=box_layout_row)   
        slider_out = widgets.interactive_output(on_slider, {'t_slider':time_slider})
        display(box_7,slider_out) 
        
    else:
        
        # when it's a single time point
        time_range = [0]

#######################################################################################
#######################################################################################    
    
def on_change(**dicts): 
    
    '''
    Input:
    - dictionary of buttons
        'button_ch_0',
        ...
        'button_ch_n',
        'drop_ch_0',
        ...
        'drop_ch_n',
        'ref_toggle'
    '''
    
    global ch_to_align
    global channels_names_selected
    global channel_luts
    global ref_channel_name
    
    # specify channels in your image file
    dict_select = [x for x in dicts if 'button' in x]
    list_values = [dicts[y] for y in dict_select]
    ch_to_align = list(np.where(list_values)[0])

    # specify names of the channels
    dict_select = [x for x in dicts if 'drop' in x]
    list_values = [dicts[y] for y in dict_select]
    channels_names_selected = list(np.array(list_values)[ch_to_align])

    # specify reference channel (from ch_to_align)
    ref_channel_name = dicts['ref_toggle']
    ref_channel_name = list_values[int(ref_channel_name[-1])-1]

#######################################################################################
#######################################################################################    
    
def on_slider(**dicts): 
    
    global time_range
    
    time_range = dicts['t_slider']
    time_range = list(range(time_range[0]-1,time_range[1])) # slider is 1-indexed
    
#######################################################################################
####################################################################################### 
def choose_all_range_f(b):
    
    global all_time
    global time_slider
    
    if (b.new == True):
    
        all_time = True
        time_slider.disabled = True
        
    else:
        
        all_time = False
        time_slider.disabled = False
#######################################################################################
####################################################################################### 
def choose_3D_f(b):
    
    global transform_choice
    
    if (b.new == True):
    
        transform_choice = '3D'
        
    else:
        
        transform_choice = '2D'

In [5]:
def create_color_table(rgb):
    
    color_table = np.zeros([256,4])
    
    color_table[:,0] = np.linspace(0,rgb[0]/255,256)
    color_table[:,1] = np.linspace(0,rgb[1]/255,256)
    color_table[:,2] = np.linspace(0,rgb[2]/255,256)
    color_table[:,3] = 1 
    
    # change to imaris friendly format
    color_table = [PW.Color(*x) for x in color_table]
    
    return color_table

#######################################################################################
####################################################################################### 

#functionn to define a list of imaris friendly color_tables
def generate_luts(channels_names_selected):
    
    channel_luts = []
    for ch in channels_names_selected:
        
        if ch == 'ch_488':
            
            color_table = create_color_table([0,255,0])
            
        elif ch == 'ch_561':
            
            color_table = create_color_table([255,0,0])
            
        elif ch == 'ch_637':
            
            color_table = create_color_table([255,0,255])
            
        elif ch == 'ch_405':
            
            color_table = create_color_table([0,0,255])
            
        else:
            
            color_table = create_color_table([255,255,255])
            
        channel_luts.append(color_table)

    return channel_luts 

#######################################################################################
####################################################################################### 

def calculate_rescale_z(z_step,im_resolution):
    
    z_step_data = im_resolution[0]*1000
    
    rescale_z = z_step_data / z_step
    
    return rescale_z

#######################################################################################
#######################################################################################  

def prepare_transforms(tmat_xy_list,tmat_xyz_list,channel_names_org,channels_names_selected,ref_channel_name,rescale_z):
    
    global transform_choice
    
    z_step_data = im_resolution[0]*1000
    
    # select transformations matching chosen channels
    if (transform_choice == '3D'):
        tmat_data_org = select_transforms(tmat_xyz_list,channel_names_org,channels_names_selected)
    else:
        tmat_data_org = select_transforms(tmat_xy_list,channel_names_org,channels_names_selected)

    # anchor transformations to the selecte channel
    ref_channel = channels_names_selected.index(ref_channel_name)
    tmat_data_anchored = anchor_transforms(tmat_data_org,ref_channel)

    # rescale in z
    if (transform_choice == '3D'):
        tmat_adjusted = z_adjust_transforms(tmat_data_anchored,rescale_z)
    else:
        tmat_adjusted = tmat_data_anchored
    
    return tmat_adjusted
    
#######################################################################################
#######################################################################################  

def select_transforms(tmat_list_org,channel_names_org,channels_names_selected):
    
    '''
    Input:
    - tmat_list_org - list of transformations from the alignment data
    - channel_names_org - order of channels in the alignment data
    - channels_names_selected - channels selected for alignment
    Output:
    - list of transforms matching the selected data
    '''
    
    tmat_data_org = []
    
    for channel_name in channels_names_selected:
        
        # check index in the original list
        channel_index_org = channel_names_org.index(channel_name)
        
        # get the rigth transform
        tmat = tmat_list_org[channel_index_org]
        
        tmat_data_org.append(tmat)
        
        
    return tmat_data_org

#######################################################################################
#######################################################################################  
def anchor_transforms(tmat_data_org,ref_channel):
    
    '''
    Input:
    - list of channel transormations
    - index of the reference channel (integer)
    Output:
    - list of channel transformations 
    anchored to the ref channel
    '''
    
    tmat_ref = tmat_data_org[ref_channel]
    
    tmat_data_anchored = []
    
    for tmat in tmat_data_org:
        
        tmat_t = tmat@np.linalg.inv(tmat_ref)
        
        tmat_data_anchored.append(tmat_t)
    
    return tmat_data_anchored
#######################################################################################
#######################################################################################  
def z_adjust_transforms(tmat_list,rescale_z):
    
    '''
    Input:
    - tmat_list - list of transforms (2D or 3D) to adjust
    - rescale_z - factor by which z-step in the data is bigger than in the reference
    Output:
    - tmat_adjusted - list of modified transforms
    '''
    
    tmat_adjusted = []
    
    for tmat in tmat_list:
        
        # check dimensionality of the tmat
        dim_2 = (tmat.shape[0] == 3)
        
        # adjust tmat to a different z-step
        if dim_2 == True:
            
            tmat[0,2] = tmat[0,2] / rescale_z
        
        else:
            
            tmat[2,3] = tmat[2,3] / rescale_z
            
            
        tmat_adjusted.append(tmat)
        
    return tmat_adjusted
#######################################################################################
#######################################################################################  
def get_selected_channels(im_path,ch_selected,time_point):
    
    im = ims(im_path)

    data_org = im[time_point,ch_selected,:,:,:]
    
    data_org = np.squeeze(data_org)
    
    return data_org

#######################################################################################
#######################################################################################  
def align_stack_3D(stack,tmat):
    
    print('3D data & 3D trannsform')
    
    stack = np.moveaxis(stack,0,2)
    
    ch_aligned = ndi.affine_transform(stack, tmat,order=1,prefilter=False)
    
    ch_aligned = np.moveaxis(ch_aligned,2,0)
    
    return ch_aligned

#######################################################################################
#######################################################################################  
def align_stack_2D(stack,tmat):
    
    print('3D data & 2D trannsform')
    
    results_list = []
    for frame in stack:
    
        ch_aligned = ndi.affine_transform(frame, tmat,order=1,prefilter=False)
        
        results_list.append(ch_aligned)
    
    ch_aligned = np.array(results_list)
    
    return ch_aligned

#######################################################################################
#######################################################################################  
def align_2D(stack,tmat):
    
    print('2D data & 2D trannsform')
    
    ch_aligned = ndi.affine_transform(stack, tmat,order=1,prefilter=False)
    
    ch_aligned = np.expand_dims(ch_aligned, axis=0)
    
    return ch_aligned

#######################################################################################
####################################################################################### 
def align_f_multi(b):
    
    global results
    global time_range
    global im_shape
    global frame_num
    
    rescale_z = calculate_rescale_z(z_step,im_resolution)
    
    tmat_adjusted = prepare_transforms(tmat_xy_list,tmat_xyz_list,channel_names_org,channels_names_selected,ref_channel_name,rescale_z)
    
    for im_path in im_path_list:
        
        if (all_time == True):
            
            im_shape,_,_ = get_metadata(im_path)
            time_range = list(range(im_shape[0]))
        
        # this function will align and save the results
        results = align_f(rescale_z,tmat_adjusted,im_path)
    
        
    print('All data registered.')
    
    return results


#######################################################################################
####################################################################################### 

def align_f(rescale_z,tmat_adjusted,im_path):
    
    global results
    
    results = []

    for time_point in time_range:

        t_array = []

        for tmat,ch_selected in zip(tmat_adjusted,ch_to_align):
            
            print(f'Processing time point {time_point+1}, channel {ch_selected}.')

            data_org = get_selected_channels(im_path,int(ch_selected),time_point)
            
            # choose proper alignment depending on data format
            if ((image_3D == True) and (transform_choice == '3D')):

                ch_aligned = align_stack_3D(data_org,tmat)
                
            elif ((image_3D == True) and (transform_choice == '2D')):
                
                ch_aligned = align_stack_2D(data_org,tmat)
                
            else:
                
                ch_aligned = align_2D(data_org,tmat)

            t_array.append(ch_aligned)

        t_array = np.array(t_array)

        results.append(t_array)
        
    print(f'{im_path} registered.')
    

    save_path = im_path.replace('.ims',f'_{transform_choice}_aligned.ims')
    save_results_ims(results,save_path,im_resolution,im_dtype,channels_names_selected)
        
    return results

#######################################################################################
####################################################################################### 
class MyCallbackClass(PW.CallbackClass):
    def __init__(self):
        self.mUserDataProgress = 0

    def RecordProgress(self, progress, total_bytes_written):
        progress100 = int(progress * 100)
        if progress100 - self.mUserDataProgress >= 5:
            self.mUserDataProgress = progress100
            print('User Progress {}, Bytes written: {}'.format(self.mUserDataProgress, total_bytes_written))
            
#######################################################################################
####################################################################################### 

def save_results_ims(results,save_path,im_resolution,im_dtype,channels_names_selected):
    
    # choose luts
    channel_luts = generate_luts(channels_names_selected)
    
    results = np.asarray(results)
    
    x = results.shape[4]
    y = results.shape[3]
    z = results.shape[2]
    c = results.shape[1]
    t = results.shape[0]

    im_extent = np.array([z,y,x])*np.array(im_resolution)

    image_size = PW.ImageSize(x=x, y=y, z=z, c=c, t=t)

    dimension_sequence = PW.DimensionSequence('x','y','z','c','t')
    sample_size = PW.ImageSize(x=1, y=1, z=1, c=1, t=1)
    block_size = image_size
    output_filename = save_path

    options = PW.Options()
    options.mNumberOfThreads = 8
    options.mCompressionAlgorithmType = PW.eCompressionAlgorithmGzipLevel2
    options.mEnableLogProgress = True


    application_name = 'PyImarisWriter'
    application_version = '0.0.7'

    callback_class = MyCallbackClass()
    converter = PW.ImageConverter(im_dtype.name, image_size, sample_size, dimension_sequence, block_size,
                                  output_filename, options, application_name, application_version, callback_class)

    num_blocks = image_size / block_size

    block_index = PW.ImageSize()
    for c in range(num_blocks.c):
        block_index.c = c
        for t in range(num_blocks.t):
            block_index.t = t
            for z in range(num_blocks.z):
                block_index.z = z
                for y in range(num_blocks.y):
                    block_index.y = y
                    for x in range(num_blocks.x):
                        block_index.x = x
                        if converter.NeedCopyBlock(block_index):
                            converter.CopyBlock(results, block_index)

    adjust_color_range = True
    image_extents = PW.ImageExtents(0, 0, 0, im_extent[2], im_extent[1], im_extent[0])

    parameters = PW.Parameters()
    color_infos = [PW.ColorInfo() for _ in range(image_size.c)]
    time_infos = [datetime.today()]
    
    for i,ch in enumerate(channels_names_selected):
        
        parameters.set_channel_name(i, ch)
        color_infos[i].set_color_table(channel_luts[i])

    # sending metadata
    converter.Finish(image_extents, parameters, time_infos, color_infos, adjust_color_range)

    converter.Destroy()
    
    print('Results have been saved.')

<div align="center"><b> <img src="dragonfly_misaligned_bw.png" alt="centered image" width = 300 /> </b></div>

In [6]:
# layout of the centered box
box_layout_col = widgets.Layout(display='flex',
                flex_flow='column',
                align_items='center',
                width='100%')

box_layout_row = widgets.Layout(display='flex',
                flex_flow='row',
                align_items='center',
                width='60%')

#################################################################
select_file_b = widgets.Button(description="Select data to align")
select_file_b.on_click(select_file_movie)

box_1 = widgets.HBox(children=[select_file_b],layout=box_layout_col)
display(box_1)

out_file = widgets.Output()
display(out_file)

#################################################################
select_align_b = widgets.Button(description="Select alignment")
select_align_b.on_click(select_file_align)

box_2 = widgets.HBox(children=[select_align_b],layout=box_layout_col)
display(box_2)

out_align = widgets.Output()
display(out_align)

#################################################################
show_options_b = widgets.Button(description="Show options")
show_options_b.on_click(show_options)
show_options_b.layout.visibility = 'hidden'

box_3 = widgets.HBox(children=[show_options_b],layout=box_layout_col)
display(box_3)

show_opt = widgets.Output()
display(show_opt)

Output()

Output()

Output()

In [7]:
all_time

True

In [8]:
time_range

NameError: name 'time_range' is not defined